### import

In [3]:
import tensorflow as tf
tf.test_is_gpu_avaliable()

AttributeError: module 'tensorflow' has no attribute 'test_is_gpu_avaliable'

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder # 범주형 수치형으로 변환
import random
import os

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from transformers import BertModel
from torch.optim import Adam

import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action = 'ignore')

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### 하이퍼 파라미터

In [5]:
CFG = {
    'EPOCHS':3,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':256,
    'SEED':41
}

### 시드 고정

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # seed 고정

### 데이터 로드

In [7]:
train = pd.read_csv('data/train.csv')
train.isnull().sum() # 결측치 확인

ID             0
Utterance      0
Speaker        0
Dialogue_ID    0
Target         0
dtype: int64

### Label encoding

In [8]:
le = LabelEncoder()
le = le.fit(train['Target'])
train['Target'] = le.transform(train['Target'])

### 훈련/검증 데이터 분할

In [9]:
valid = train[train['Dialogue_ID'].isin([i for i in range(1016,1039)])].reset_index(drop = True)
train = train[~train['Dialogue_ID'].isin([i for i in range(1016,1039)])].reset_index(drop = True)

train_len = len(train)
val_len = len(valid)

print(train_len, val_len)

9725 264


### Tokenizer Define

In [10]:
tokenizers = BertTokenizer.from_pretrained('bert-base-cased')

### CustomDataset

In [11]:
class CustomDataset(Dataset):
  
    def __init__(self, data, mode = "train"):
        self.dataset = data
        self.tokenizer = tokenizers
        self.mode = mode
        
    def __len__(self):
        return len(self.dataset)
  
    def __getitem__(self, idx):
        text = self.dataset['Utterance'][idx]
        inputs = self.tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
        input_ids = inputs['input_ids'][0]
        token_type_ids = inputs['token_type_ids'][0]
        attention_mask = inputs['attention_mask'][0]
    
        if self.mode == "train":
            y = self.dataset['Target'][idx]
            return input_ids, token_type_ids, attention_mask, y
        else:
            return input_ids, token_type_ids, attention_mask

In [12]:
train = CustomDataset(train, mode = "train")
valid = CustomDataset(valid, mode = "train")

train_dataloader = torch.utils.data.DataLoader(train, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_dataloader = torch.utils.data.DataLoader(valid, batch_size=CFG['BATCH_SIZE'], shuffle=True)

### Model Define

In [13]:
class BaseModel(nn.Module):
    
    def __init__(self, dropout = 0.2, num_classes = len(le.classes_)):
        
        super(BaseModel, self).__init__()
        
        self.bert = BertModel.from_pretrained("bert-base-cased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids = input_id, attention_mask = mask, return_dict = False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        
        return final_layer        

### Train

In [14]:
def competition_metric(true, pred):
    return f1_score(true, pred, average = "macro")

def validation(model, criterion, test_loader, device):
    model.eval()
    
    val_loss = []
    model_preds = []
    true_labels = []
    with torch.no_grad():
        for input_ids, token_type_ids, attention_mask, valid_label in tqdm(test_loader):
            valid_label = valid_label.to(device)
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)
            
            output = model(input_id, mask)
            
            batch_loss = criterion(output, valid_label.long())
            val_loss.append(batch_loss.item())
            
            model_preds += output.argmax(1).detach().cpu().numpy.tolist()
            true_labels += valid_label.detach().cpu().numpy().tolist()
        val_f1 = competition_metric(true_labels, model_preds)
    return val_loss, val_f1

In [15]:
def train(model, optimizer, train_loader, test_loader, device):
    
    model.to(device)
    
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = "None"
    for epoch_num in range(CFG['EPOCHS']):
        
        model.train()
        train_loss = []
        for input_ids, token_type_ids, attention_mask, train_label in tqdm(train_loader):
            optimizer.zero_grad()
            
            train_label = train_label.to(device)
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)
            
            output = model(input_id, mask)
            
            batch_loss = criterion(output, train_label.long())
            train_loss.append(batch_loss.item())
            
            batch_loss.backward()
            optimizer.step()
            
        val_loss, val_score = validation(model, criterion, test_loader, device)
        print(f'Epoch [{epoch_num}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if best_score < val_score:
            best_model = model
            best_score = val_score
            
    return best_model
        

### RUN

In [16]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params=model.parameters(), lr = CFG["LEARNING_RATE"])

infer_model = train(model, optimizer, train_dataloader, val_dataloader, device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/38 [00:00<?, ?it/s]

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3221225472 bytes.

### Inference

In [18]:
test = pd.read_csv('data/test.csv')

test = CustomDataset(test, mode = "test")
test_dataloader = torch.utils.data.DataLoader(test, batch_size = CFG['BATCH_SIZE'], shuffle = False)

def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    test_predict = []
    for input_ids, token_type_ids, attention_mask in tqdm(test_loader):
        input_id = input_ids.to(device)
        mask = attention_mask.to(device)
        y_pred = model(input_id, mask)
        test_predict += y_pred.argmax(1).detach().cpu().numpy().tolist()
    print('Done.')
    return test_predict

In [19]:
preds = inference(infer_model, test_dataloader, device)

NameError: name 'infer_model' is not defined

In [ ]:
preds = le.inverse_transform(preds)

In [ ]:
submit = pd.read_csv('data/sample_submission.csv')
submit['Target'] = preds
submit.head()

In [ ]:
submit.to_csv('./submit.csv', index=False)
